# Strands Agents with AgentCore Deployment
Learn how to build agents with Strands framework and deploy them to AWS Bedrock AgentCore

## Setup

Install required packages:
```bash
pip install strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit
```

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator
from bedrock_agentcore import BedrockAgentCoreApp

## 1. Basic Strands Agent

In [ ]:
# Simple agent with calculator tool
agent = Agent(tools=[calculator])
response = agent("What is 25 * 47?")
print(response)

## 2. Custom Tools

In [ ]:
@tool
def get_weather(city: str) -> dict:
    """Get current weather for a city."""
    # Mock implementation
    return {"city": city, "temp": 72, "condition": "sunny"}

@tool
def get_forecast(city: str, days: int = 3) -> dict:
    """Get weather forecast for a city."""
    return {"city": city, "forecast": ["sunny", "cloudy", "rainy"][:days]}

weather_agent = Agent(
    system_prompt="You are a weather assistant.",
    tools=[get_weather, get_forecast]
)

response = weather_agent("What's the weather in Seattle and give me a 5-day forecast?")
print(response)

## 3. Using Different Models

In [ ]:
# Amazon Bedrock Nova
nova_model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    region="us-east-1",
    temperature=0.7,
    streaming=True
)

nova_agent = Agent(
    model=nova_model,
    system_prompt="You are a helpful assistant.",
    tools=[calculator]
)

response = nova_agent("Calculate the compound interest on $10,000 at 5% for 3 years")
print(response)

## 4. Multi-Agent Pattern: Agents as Tools

In [ ]:
# Specialist agents as tools
@tool
def research_assistant(query: str) -> str:
    """Research specialist for gathering information."""
    research_agent = Agent(
        system_prompt="You are a research specialist. Provide factual information.",
        tools=[calculator]
    )
    return str(research_agent(query))

@tool
def writer_assistant(content: str) -> str:
    """Writing specialist for creating polished content."""
    writer_agent = Agent(
        system_prompt="You are a professional writer. Create clear, engaging content."
    )
    return str(writer_agent(f"Write a summary: {content}"))

# Orchestrator agent
orchestrator = Agent(
    system_prompt="You coordinate between research and writing specialists.",
    tools=[research_assistant, writer_assistant]
)

response = orchestrator("Research AI trends and write a brief summary")
print(response)

## 5. Multi-Agent Pattern: Swarm

In [ ]:
from strands.multiagent import Swarm

# Create specialized agents
researcher = Agent(
    name="researcher",
    system_prompt="Research and gather information, then hand off to analyst.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0"),
    tools=[calculator]
)

analyst = Agent(
    name="analyst",
    system_prompt="Analyze data and hand off to writer.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0")
)

writer = Agent(
    name="writer",
    system_prompt="Create final report based on analysis.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0")
)

# Configure swarm
swarm = Swarm(
    agents=[researcher, analyst, writer],
    max_handoffs=2,
    max_iterations=3,
    execution_timeout=300.0,
    node_timeout=60.0
)

result = swarm("Analyze the benefits of cloud computing")
print(f"Final: {result.final_response}")
print(f"Path: {[node.node_id for node in result.node_history]}")

## 6. Wrap Strands Agent for AgentCore Deployment

In [ ]:
# Create AgentCore app
app = BedrockAgentCoreApp()

# Create your Strands agent
my_agent = Agent(
    system_prompt="You are a helpful assistant with weather and calculation capabilities.",
    tools=[get_weather, get_forecast, calculator]
)

@app.entrypoint
def invoke(payload):
    """AgentCore entrypoint that wraps Strands agent."""
    prompt = payload.get("prompt", "Hello!")
    response = my_agent(prompt)
    return {"result": str(response)}

# Test locally
if __name__ == "__main__":
    # This would run the agent locally on port 8080
    # app.run()
    
    # For notebook testing
    test_payload = {"prompt": "What's the weather in Boston?"}
    result = invoke(test_payload)
    print(result)

## 7. Save Agent for Deployment

Save the agent code to a file for AgentCore deployment:

In [ ]:
agent_code = '''
from strands import Agent, tool
from strands_tools import calculator
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@tool
def get_weather(city: str) -> dict:
    """Get current weather for a city."""
    return {"city": city, "temp": 72, "condition": "sunny"}

@tool
def get_forecast(city: str, days: int = 3) -> dict:
    """Get weather forecast for a city."""
    return {"city": city, "forecast": ["sunny", "cloudy", "rainy"][:days]}

agent = Agent(
    system_prompt="You are a helpful weather and calculation assistant.",
    tools=[get_weather, get_forecast, calculator]
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    response = agent(prompt)
    return {"result": str(response)}

if __name__ == "__main__":
    app.run()
'''

with open('strands_weather_agent.py', 'w') as f:
    f.write(agent_code)

print("✓ Agent saved to strands_weather_agent.py")

## 8. Deploy to AgentCore

Run these commands in terminal:

```bash
# Configure
agentcore configure -e strands_weather_agent.py

# Deploy
agentcore deploy

# Test
agentcore invoke '{"prompt": "What is the weather in Seattle?"}'

# Clean up
agentcore destroy
```

## 9. Advanced: Multi-Agent Swarm for AgentCore

In [ ]:
advanced_agent_code = '''
from strands import Agent
from strands.models import BedrockModel
from strands.multiagent import Swarm
from strands_tools import calculator
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Create swarm agents
researcher = Agent(
    name="researcher",
    system_prompt="Research and gather information.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0"),
    tools=[calculator]
)

analyst = Agent(
    name="analyst",
    system_prompt="Analyze data and provide insights.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0")
)

writer = Agent(
    name="writer",
    system_prompt="Create clear, concise reports.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0")
)

swarm = Swarm(
    agents=[researcher, analyst, writer],
    max_handoffs=2,
    max_iterations=2,
    execution_timeout=180.0
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    result = swarm(prompt)
    return {
        "result": str(result.final_response),
        "path": [node.node_id for node in result.node_history]
    }

if __name__ == "__main__":
    app.run()
'''

with open('strands_swarm_agent.py', 'w') as f:
    f.write(advanced_agent_code)

print("✓ Swarm agent saved to strands_swarm_agent.py")

## 10. Invoke Deployed Agent Programmatically

In [ ]:
import boto3
import json
import uuid

def invoke_agentcore_agent(agent_arn: str, prompt: str):
    """Invoke a deployed AgentCore agent."""
    client = boto3.client('bedrock-agentcore')
    
    payload = json.dumps({"prompt": prompt}).encode()
    
    response = client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=payload,
        qualifier="DEFAULT"
    )
    
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    
    return json.loads(''.join(content))

# Example usage (replace with your agent ARN)
# agent_arn = "arn:aws:bedrock-agentcore:us-east-1:123456789012:agent-runtime/my-agent"
# result = invoke_agentcore_agent(agent_arn, "What's the weather in Boston?")
# print(result)

## Summary

### Key Concepts

1. **Strands Agent**: Lightweight, model-driven agent framework
2. **Custom Tools**: Use `@tool` decorator to create agent capabilities
3. **Multi-Agent Patterns**: Agents as Tools, Swarm, Graph, Workflow
4. **AgentCore Integration**: Wrap Strands agents with `BedrockAgentCoreApp`
5. **Deployment**: Use `agentcore` CLI for serverless deployment

### Workflow

```
Build Strands Agent → Wrap with AgentCore → Deploy → Invoke
```

### Benefits

- **Strands**: Simple, flexible, model-driven agent development
- **AgentCore**: Production-ready serverless infrastructure
- **Combined**: Best of both worlds - easy development + enterprise deployment

### Next Steps

1. Experiment with different multi-agent patterns
2. Add more sophisticated tools
3. Deploy to AgentCore for production use
4. Monitor with CloudWatch and AgentCore Observability
5. Scale with AgentCore's serverless infrastructure